In [1]:
!pip install --upgrade paddlenlp

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple

[notice] A new release of pip available: 22.1.2 -> 22.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import os
import paddle
import paddlenlp

In [5]:
# 自定义数据集
import re

from paddlenlp.datasets import load_dataset

# 清洗无效字符
def clean_text(text):
    text = text.replace("\r", "").replace("\n", "")
    text = re.sub(r"\\n\n", ".", text)
    return text

# 定义读取数据集函数
def read_custom_data(filepath):
    f = open(filepath)
    next(f)
    while True:
        line = f.readline()
        if not line:
            break
        data = line.strip().split('\t')
        labels = [float(d) for d in data[2:]]
        yield {"Argument ID": data[0], "sentence":clean_text(data[1]),"labels": labels}
    f.close()

def read_custom_data_test(filepath):
    f = open(filepath)
    next(f)
    while True:
        line = f.readline()
        if not line:
            break
        data = line.strip().split('\t')
        yield {"Argument ID": data[0], "sentence":clean_text(data[1]),"labels":[]}
    f.close()

In [6]:
%cd /home/aistudio/data

/home/aistudio/data


In [7]:
# load_dataset()创建数据集
# lazy=False，数据集返回为MapDataset类型
# 对训练集和验证集进行预处理
train_ds = load_dataset(read_custom_data, filepath='train.tsv', lazy=False) 
valid_ds = load_dataset(read_custom_data, filepath='validation.tsv', lazy=False)
validZhihu_ds = load_dataset(read_custom_data, filepath='zhihu_validation.tsv', lazy=False)
test_ds = load_dataset(read_custom_data_test, filepath='test.tsv', lazy=False) 

In [8]:
# 加载中文ERNIE 3.0预训练模型和分词器
from paddlenlp.transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "ernie-2.0-large-en"   # ERNIE2.0 模型
num_classes = 20  # 20分类任务
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_classes=num_classes)
tokenizer = AutoTokenizer.from_pretrained(model_name)

[2022-12-29 14:07:11,962] [    INFO] - We are using <class 'paddlenlp.transformers.ernie.modeling.ErnieForSequenceClassification'> to load 'ernie-2.0-large-en'.
[2022-12-29 14:07:11,966] [    INFO] - Model config ErnieConfig {
  "attention_probs_dropout_prob": 0.1,
  "enable_recompute": false,
  "fuse": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19"
  },
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_10": 10,
    "LABEL_11": 11,
    "LABEL_12": 12,
 

In [9]:
import functools
import numpy as np

from paddle.io import DataLoader, BatchSampler
from paddlenlp.data import DataCollatorWithPadding

# 数据预处理函数，利用分词器将文本转化为整数序列
def preprocess_function(examples, tokenizer, max_seq_length):
    result = tokenizer(text=examples["sentence"], max_seq_len=max_seq_length)
    result["labels"] = examples["labels"]
    return result

trans_func = functools.partial(preprocess_function, tokenizer=tokenizer, max_seq_length=128)
train_ds = train_ds.map(trans_func)
valid_ds = valid_ds.map(trans_func)
validZhihu_ds = validZhihu_ds.map(trans_func)
test_ds = test_ds.map(trans_func)

# collate_fn函数构造，将不同长度序列充到批中数据的最大长度，再将数据堆叠
collate_fn = DataCollatorWithPadding(tokenizer)

# 定义BatchSampler，选择批大小和是否随机乱序，进行DataLoader
train_batch_sampler = BatchSampler(train_ds, batch_size=32, shuffle=True)
valid_batch_sampler = BatchSampler(valid_ds, batch_size=16, shuffle=False)
validZhihu_batch_sampler = BatchSampler(validZhihu_ds, batch_size=16, shuffle=False)
test_batch_sampler = BatchSampler(test_ds, batch_size=16, shuffle=False)

train_data_loader = DataLoader(dataset=train_ds, batch_sampler=train_batch_sampler, collate_fn=collate_fn)
valid_data_loader = DataLoader(dataset=valid_ds, batch_sampler=valid_batch_sampler, collate_fn=collate_fn)
validZhihu_data_loader = DataLoader(dataset=validZhihu_ds, batch_sampler=validZhihu_batch_sampler, collate_fn=collate_fn)
test_data_loader = DataLoader(dataset=test_ds, batch_sampler=test_batch_sampler, collate_fn=collate_fn)

In [10]:
import numpy as np
import sklearn
from sklearn.metrics import roc_auc_score, f1_score, precision_score, recall_score
from paddle.metric import Metric

# 自定义MultiLabelReport评价指标
class MultiLabelReport(Metric):
    """
    AUC and F1 Score for multi-label text classification task.
    """

    def __init__(self, name='MultiLabelReport', average='macro'):
        super(MultiLabelReport, self).__init__()
        self.average = average
        self._name = name
        self.reset()

    # def f1_score(self, y_prob):
    #     '''
    #     Returns the f1 score by searching the best threshhold
    #     '''
    #     best_score = 0
    #     for threshold in [i * 0.01 for i in range(100)]:
    #         self.y_pred = y_prob > threshold
    #         score = sklearn.metrics.f1_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
    #         if score > best_score:
    #             best_score = score
    #             precison = precision_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
    #             recall = recall_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
    #     return best_score, precison, recall

    def f1_score(self, y_prob):
        '''
        Returns the f1 score by searching the best threshhold
        '''
        thresholds =0
        self.y_pred = y_prob > thresholds
        score = sklearn.metrics.f1_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        precison = precision_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        recall = recall_score(y_pred=self.y_pred, y_true=self.y_true, average=self.average)
        return score, precison, recall

    def reset(self):
        """
        Resets all of the metric state.
        """
        self.y_prob = None
        self.y_true = None

    def update(self, probs, labels):
        if self.y_prob is not None:
            self.y_prob = np.append(self.y_prob, probs.numpy(), axis=0)
        else:
            self.y_prob = probs.numpy()
        if self.y_true is not None:
            self.y_true = np.append(self.y_true, labels.numpy(), axis=0)
        else:
            self.y_true = labels.numpy()

    def accumulate(self):
        # auc = roc_auc_score(
        #     y_score=self.y_prob, y_true=self.y_true, average=self.average)
        f1_score, precison, recall = self.f1_score(y_prob=self.y_prob)
        # return auc, f1_score, precison, recall
        return f1_score, precison, recall
    

    def name(self):
        """
        Returns metric name
        """
        return self._name

In [11]:
import numpy as np
def multilabel_categorical_crossentropy(y_true, y_pred):
    """多标签分类的交叉熵
    说明：y_true和y_pred的shape一致，y_true的元素非0即1，
         1表示对应的类为目标类，0表示对应的类为非目标类。
    警告：请保证y_pred的值域是全体实数，换言之一般情况下y_pred
         不用加激活函数，尤其是不能加sigmoid或者softmax！预测
         阶段则输出y_pred大于0的类。如有疑问，请仔细阅读并理解
         本文。
    假如类别总数为10
    label ：[0,1,0,0,0,0,0,0,0,1]  代表条数据被标注为 2,10 属于 2类也属于10类
    输出也为10类别 输出维度也为10。
    类别从1位置开始0位置代表阈值s就是输出的维度第一个位置是阈值预测
    目标类的分数都大于s，非目标类的分数都小于s
    这里阈值s默认为0故而可忽略只要类从1开始就可
    """
    y_pred = (1 - 2 * y_true) * y_pred
    y_pred_neg = y_pred - y_true * 1e12
    y_pred_pos = y_pred - (1 - y_true) * 1e12


    zeros = paddle.zeros_like(y_pred[..., :1])

    y_pred_neg = paddle.concat((y_pred_neg, zeros), axis=-1)
    y_pred_pos = paddle.concat((y_pred_pos, zeros), axis=-1)


    neg_loss = paddle.logsumexp(y_pred_neg, axis=-1)
    pos_loss = paddle.logsumexp(y_pred_pos, axis=-1)
    return neg_loss + pos_loss

In [12]:
import time
import paddle.nn.functional as F

# AdamW优化器、交叉熵损失函数、自定义MultiLabelReport评价指标
optimizer = paddle.optimizer.AdamW(learning_rate=4e-5, parameters=model.parameters(), weight_decay=0.01)
criterion = multilabel_categorical_crossentropy
metric = MultiLabelReport()

In [13]:
label_vocab = ["Self-direction: thought","Self-direction: action","Stimulation","Hedonism","Achievement","Power: dominance","Power: resources","Face","Security: personal","Security: societal","Tradition","Conformity: rules","Conformity: interpersonal","Humility","Benevolence: caring","Benevolence: dependability","Universalism: concern","Universalism: nature","Universalism: tolerance","Universalism: objectivity"]

In [14]:
import paddle
import numpy as np
import paddle.nn.functional as F

# 构建验证集evaluate函数
@paddle.no_grad()
def evaluate(model, criterion, metric, data_loader, label_vocab, if_return_results=True):
    model.eval()
    metric.reset()
    losses = []
    results = []
    for batch in data_loader:
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']
        logits = model(input_ids, token_type_ids)
        loss = criterion(labels,logits)
        # probs = F.sigmoid(logits)
        probs = logits
        loss = loss.mean()
        losses.append(loss.numpy())
        metric.update(probs, labels)
        if if_return_results:
            probs = probs.tolist()
            for prob in probs:
                result = []
                for c, pred in enumerate(prob):
                    if pred > 0:
                        result.append(label_vocab[c])
                        # result.append(str(c))
                results.append(','.join(result))

    # auc, f1_score, precison, recall = metric.accumulate()
    f1_score, precison, recall = metric.accumulate()
    print("eval loss: %.5f, f1 score: %.5f, precison: %.5f, recall: %.5f" %
          (np.mean(losses), f1_score, precison, recall))
    model.train()
    metric.reset()
    if if_return_results:
        return results
    else:
        return f1_score

In [15]:
%cd /home/aistudio/model

/home/aistudio/model


In [16]:
epochs = 20 # 训练轮次
ckpt_dir = "ernie2.0_ckpt" # 训练过程中保存模型参数的文件夹
save_dir2 = "ernie2.0_ckpt_zhihu"

global_step = 0  # 迭代次数
tic_train = time.time()
best_f1_score = 0
best_f1_score2 = 0

# 模型训练
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率、f1分数
        logits = model(input_ids, token_type_ids)
        
        loss = criterion(labels, logits)
        loss = loss.mean()
        probs = logits
        #probs = F.sigmoid(logits)
        metric.update(probs, labels)
        
        # auc, f1_score, _,  _= metric.accumulate()
        f1_score, _,  _= metric.accumulate()


        # 每迭代100次，打印损失函数值、准确率、f1分数、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        #每迭代40次，评估当前训练的模型、保存当前最佳模型参数和分词器的词表等
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, valid_data_loader, label_vocab, if_return_results=False)
            eval_f1_score2 = evaluate(model, criterion, metric, validZhihu_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)
            if eval_f1_score2 > best_f1_score2:
                best_f1_score2 = eval_f1_score2
                model.save_pretrained(save_dir2)
                tokenizer.save_pretrained(save_dir2)

global step 10, epoch: 1, batch: 10, loss: 4.10663, f1 score: 0.21181, speed: 1.63 step/s
global step 20, epoch: 1, batch: 20, loss: 4.07766, f1 score: 0.19203, speed: 1.72 step/s
global step 30, epoch: 1, batch: 30, loss: 4.06239, f1 score: 0.17248, speed: 1.53 step/s
global step 40, epoch: 1, batch: 40, loss: 3.94547, f1 score: 0.16890, speed: 1.61 step/s
eval loss: 3.91192, f1 score: 0.10681, precison: 0.23412, recall: 0.11292
eval loss: 3.69470, f1 score: 0.06602, precison: 0.07659, recall: 0.07246


[2022-12-29 11:16:36,317] [    INFO] - Configuration saved in ernie2.0_ckpt/config.json
[2022-12-29 11:16:39,506] [    INFO] - tokenizer config file saved in ernie2.0_ckpt/tokenizer_config.json
[2022-12-29 11:16:39,509] [    INFO] - Special tokens file saved in ernie2.0_ckpt/special_tokens_map.json
[2022-12-29 11:16:39,515] [    INFO] - Configuration saved in ernie2.0_ckpt_zhihu/config.json
[2022-12-29 11:16:42,619] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_zhihu/tokenizer_config.json
[2022-12-29 11:16:42,623] [    INFO] - Special tokens file saved in ernie2.0_ckpt_zhihu/special_tokens_map.json


global step 50, epoch: 1, batch: 50, loss: 3.82056, f1 score: 0.22150, speed: 0.42 step/s
global step 60, epoch: 1, batch: 60, loss: 3.73289, f1 score: 0.23713, speed: 1.67 step/s
global step 70, epoch: 1, batch: 70, loss: 3.80777, f1 score: 0.26480, speed: 1.54 step/s
global step 80, epoch: 1, batch: 80, loss: 3.58494, f1 score: 0.27662, speed: 1.81 step/s
eval loss: 3.68114, f1 score: 0.26435, precison: 0.42329, recall: 0.22789
eval loss: 3.47785, f1 score: 0.18896, precison: 0.24290, recall: 0.19614


[2022-12-29 11:17:18,011] [    INFO] - Configuration saved in ernie2.0_ckpt/config.json
[2022-12-29 11:17:34,000] [    INFO] - tokenizer config file saved in ernie2.0_ckpt/tokenizer_config.json
[2022-12-29 11:17:34,004] [    INFO] - Special tokens file saved in ernie2.0_ckpt/special_tokens_map.json
[2022-12-29 11:17:34,009] [    INFO] - Configuration saved in ernie2.0_ckpt_zhihu/config.json
[2022-12-29 11:17:46,169] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_zhihu/tokenizer_config.json
[2022-12-29 11:17:46,173] [    INFO] - Special tokens file saved in ernie2.0_ckpt_zhihu/special_tokens_map.json


global step 90, epoch: 1, batch: 90, loss: 3.74324, f1 score: 0.29425, speed: 0.22 step/s
global step 100, epoch: 1, batch: 100, loss: 3.58679, f1 score: 0.30519, speed: 1.65 step/s
global step 110, epoch: 1, batch: 110, loss: 3.37050, f1 score: 0.31331, speed: 1.62 step/s
global step 120, epoch: 1, batch: 120, loss: 3.63459, f1 score: 0.31910, speed: 1.72 step/s
eval loss: 3.61151, f1 score: 0.31005, precison: 0.54439, recall: 0.26087
eval loss: 3.40661, f1 score: 0.23042, precison: 0.30289, recall: 0.24682


...

global step 2490, epoch: 15, batch: 124, loss: 0.26813, f1 score: 0.99123, speed: 0.23 step/s
global step 2500, epoch: 15, batch: 134, loss: 0.37820, f1 score: 0.98299, speed: 1.78 step/s
global step 2510, epoch: 15, batch: 144, loss: 0.29995, f1 score: 0.98606, speed: 1.59 step/s
global step 2520, epoch: 15, batch: 154, loss: 0.34441, f1 score: 0.98491, speed: 1.72 step/s
eval loss: 6.50462, f1 score: 0.48911, precison: 0.53623, recall: 0.47334
eval loss: 6.28880, f1 score: 0.34251, precison: 0.28111, recall: 0.47931


[2022-12-29 12:00:54,927] [    INFO] - Configuration saved in ernie2.0_ckpt/config.json
[2022-12-29 12:01:09,544] [    INFO] - tokenizer config file saved in ernie2.0_ckpt/tokenizer_config.json
[2022-12-29 12:01:09,548] [    INFO] - Special tokens file saved in ernie2.0_ckpt/special_tokens_map.json


global step 2530, epoch: 15, batch: 164, loss: 0.47579, f1 score: 0.98157, speed: 0.31 step/s
global step 2540, epoch: 16, batch: 5, loss: 0.43891, f1 score: 0.98233, speed: 1.66 step/s
global step 2550, epoch: 16, batch: 15, loss: 0.34689, f1 score: 0.98204, speed: 1.66 step/s
global step 2560, epoch: 16, batch: 25, loss: 0.42079, f1 score: 0.98301, speed: 1.64 step/s
eval loss: 6.65712, f1 score: 0.48505, precison: 0.51982, recall: 0.48329
eval loss: 6.39782, f1 score: 0.34724, precison: 0.28115, recall: 0.51078
global step 2570, epoch: 16, batch: 35, loss: 0.25801, f1 score: 0.98461, speed: 0.55 step/s
global step 2580, epoch: 16, batch: 45, loss: 0.24862, f1 score: 0.98766, speed: 1.62 step/s
global step 2590, epoch: 16, batch: 55, loss: 0.21864, f1 score: 0.98727, speed: 1.64 step/s
global step 2600, epoch: 16, batch: 65, loss: 0.50354, f1 score: 0.98738, speed: 1.53 step/s
eval loss: 6.60937, f1 score: 0.48753, precison: 0.51832, recall: 0.48009
eval loss: 6.32857, f1 score: 0.34

[2022-12-29 12:08:24,610] [    INFO] - Configuration saved in ernie2.0_ckpt/config.json
[2022-12-29 12:08:37,585] [    INFO] - tokenizer config file saved in ernie2.0_ckpt/tokenizer_config.json
[2022-12-29 12:08:37,588] [    INFO] - Special tokens file saved in ernie2.0_ckpt/special_tokens_map.json


global step 3010, epoch: 18, batch: 137, loss: 0.32662, f1 score: 0.98742, speed: 0.33 step/s
global step 3020, epoch: 18, batch: 147, loss: 0.40685, f1 score: 0.98560, speed: 1.63 step/s
global step 3030, epoch: 18, batch: 157, loss: 0.26224, f1 score: 0.98320, speed: 1.70 step/s
global step 3040, epoch: 18, batch: 167, loss: 0.26660, f1 score: 0.98207, speed: 1.59 step/s
eval loss: 7.03179, f1 score: 0.48569, precison: 0.54195, recall: 0.46862
eval loss: 6.81358, f1 score: 0.31031, precison: 0.25938, recall: 0.42364
global step 3050, epoch: 19, batch: 8, loss: 0.18939, f1 score: 0.99327, speed: 0.57 step/s
global step 3060, epoch: 19, batch: 18, loss: 0.23990, f1 score: 0.99162, speed: 1.80 step/s
global step 3070, epoch: 19, batch: 28, loss: 0.24451, f1 score: 0.99055, speed: 1.65 step/s
global step 3080, epoch: 19, batch: 38, loss: 0.23496, f1 score: 0.99075, speed: 1.60 step/s
eval loss: 7.21831, f1 score: 0.48455, precison: 0.52628, recall: 0.48090
eval loss: 6.91323, f1 score: 0

In [17]:
# 模型加载
# 加载最后的模型参数
model.set_dict(paddle.load('ernie2.0_ckpt/model_state.pdparams'))

# 加载之前训练好的模型参数
# model.set_dict(paddle.load('/home/aistudio/work/model_state.pdparams'))

# 模型验证
print("ERNIE 3.0 在20分类验证集的最佳表现：", end= " ")
results1 = evaluate(model, criterion, metric, valid_data_loader, label_vocab)
print("ERNIE 2.0 在20分类知乎验证集的最佳表现：", end= " ")
results = evaluate(model, criterion, metric, validZhihu_data_loader, label_vocab)

ERNIE 3.0 在20分类验证集的最佳表现： eval loss: 7.01792, f1 score: 0.49189, precison: 0.52654, recall: 0.48107
ERNIE 2.0 在20分类知乎验证集的最佳表现： eval loss: 6.95169, f1 score: 0.30715, precison: 0.26447, recall: 0.40759


In [ ]:
#接下来调低学习率继续训练

In [18]:
optimizer = paddle.optimizer.AdamW(learning_rate=2e-5, parameters=model.parameters(), weight_decay=0.01)
epochs = 7 # 训练轮次
ckpt_dir = "ernie2.0_ckpt_2" # 训练过程中保存模型参数的文件夹
save_dir2 = "ernie2.0_ckpt_zhihu_2"

global_step = 0  # 迭代次数
tic_train = time.time()
best_f1_score = 0.48538
best_f1_score2 = 0.33237
# 模型训练
for epoch in range(1, epochs + 1):
    for step, batch in enumerate(train_data_loader, start=1):
        input_ids, token_type_ids, labels = batch['input_ids'], batch['token_type_ids'], batch['labels']

        # 计算模型输出、损失函数值、分类概率值、准确率、f1分数
        logits = model(input_ids, token_type_ids)
        
        loss = criterion(labels, logits)
        loss = loss.mean()
        probs = logits
        #probs = F.sigmoid(logits)
        metric.update(probs, labels)
        
        # auc, f1_score, _,  _= metric.accumulate()
        f1_score, _,  _= metric.accumulate()


        # 每迭代100次，打印损失函数值、准确率、f1分数、计算速度
        global_step += 1
        if global_step % 10 == 0:
            print(
                "global step %d, epoch: %d, batch: %d, loss: %.5f, f1 score: %.5f, speed: %.2f step/s"
                % (global_step, epoch, step, loss, f1_score,
                    10 / (time.time() - tic_train)))
            tic_train = time.time()
        
        # 反向梯度回传，更新参数
        loss.backward()
        optimizer.step()
        optimizer.clear_grad()
        
        #每迭代40次，评估当前训练的模型、保存当前最佳模型参数和分词器的词表等
        if global_step % 40 == 0:
            save_dir = ckpt_dir
            if not os.path.exists(save_dir):
                os.makedirs(save_dir)
            eval_f1_score = evaluate(model, criterion, metric, valid_data_loader, label_vocab, if_return_results=False)
            eval_f1_score2 = evaluate(model, criterion, metric, validZhihu_data_loader, label_vocab, if_return_results=False)
            if eval_f1_score > best_f1_score:
                best_f1_score = eval_f1_score
                model.save_pretrained(save_dir)
                tokenizer.save_pretrained(save_dir)
            if eval_f1_score2 > best_f1_score2:
                best_f1_score2 = eval_f1_score2
                model.save_pretrained(save_dir2)
                tokenizer.save_pretrained(save_dir2)

global step 10, epoch: 1, batch: 10, loss: 0.21115, f1 score: 0.98385, speed: 1.66 step/s
global step 20, epoch: 1, batch: 20, loss: 0.16292, f1 score: 0.98723, speed: 1.73 step/s
global step 30, epoch: 1, batch: 30, loss: 0.21903, f1 score: 0.98882, speed: 1.63 step/s
global step 40, epoch: 1, batch: 40, loss: 0.31735, f1 score: 0.98897, speed: 1.77 step/s
eval loss: 7.06932, f1 score: 0.48287, precison: 0.52517, recall: 0.46811
eval loss: 6.89580, f1 score: 0.33898, precison: 0.28671, recall: 0.45758


[2022-12-29 12:28:17,567] [    INFO] - Configuration saved in ernie2.0_ckpt_zhihu_2/config.json
[2022-12-29 12:28:21,176] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_zhihu_2/tokenizer_config.json
[2022-12-29 12:28:21,180] [    INFO] - Special tokens file saved in ernie2.0_ckpt_zhihu_2/special_tokens_map.json


global step 50, epoch: 1, batch: 50, loss: 0.26855, f1 score: 0.98274, speed: 0.47 step/s
global step 60, epoch: 1, batch: 60, loss: 0.36780, f1 score: 0.98395, speed: 1.62 step/s
global step 70, epoch: 1, batch: 70, loss: 0.14001, f1 score: 0.98685, speed: 1.58 step/s
global step 80, epoch: 1, batch: 80, loss: 0.42516, f1 score: 0.98601, speed: 1.70 step/s
eval loss: 7.15546, f1 score: 0.47923, precison: 0.53627, recall: 0.45987
eval loss: 7.08550, f1 score: 0.36831, precison: 0.31563, recall: 0.48252


[2022-12-29 12:28:57,495] [    INFO] - Configuration saved in ernie2.0_ckpt_zhihu_2/config.json
[2022-12-29 12:29:10,478] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_zhihu_2/tokenizer_config.json
[2022-12-29 12:29:10,481] [    INFO] - Special tokens file saved in ernie2.0_ckpt_zhihu_2/special_tokens_map.json


global step 90, epoch: 1, batch: 90, loss: 0.24385, f1 score: 0.98606, speed: 0.33 step/s
global step 100, epoch: 1, batch: 100, loss: 0.15140, f1 score: 0.98754, speed: 1.85 step/s
global step 110, epoch: 1, batch: 110, loss: 0.11361, f1 score: 0.98634, speed: 1.65 step/s
global step 120, epoch: 1, batch: 120, loss: 0.16482, f1 score: 0.98783, speed: 1.78 step/s
eval loss: 7.11093, f1 score: 0.47989, precison: 0.52526, recall: 0.45943
eval loss: 6.98985, f1 score: 0.34697, precison: 0.33055, recall: 0.44697
global step 130, epoch: 1, batch: 130, loss: 0.23779, f1 score: 0.98184, speed: 0.55 step/s
global step 140, epoch: 1, batch: 140, loss: 0.27784, f1 score: 0.98685, speed: 1.68 step/s
global step 150, epoch: 1, batch: 150, loss: 0.15618, f1 score: 0.98787, speed: 1.78 step/s
global step 160, epoch: 1, batch: 160, loss: 0.22180, f1 score: 0.98697, speed: 1.57 step/s
eval loss: 7.04342, f1 score: 0.47451, precison: 0.53075, recall: 0.45275
eval loss: 6.96310, f1 score: 0.35033, preci

[2022-12-29 12:32:45,032] [    INFO] - Configuration saved in ernie2.0_ckpt_2/config.json
[2022-12-29 12:32:48,565] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_2/tokenizer_config.json
[2022-12-29 12:32:48,570] [    INFO] - Special tokens file saved in ernie2.0_ckpt_2/special_tokens_map.json


global step 330, epoch: 2, batch: 161, loss: 0.42477, f1 score: 0.98692, speed: 0.47 step/s
global step 340, epoch: 3, batch: 2, loss: 0.17821, f1 score: 0.98679, speed: 1.51 step/s
global step 350, epoch: 3, batch: 12, loss: 0.27965, f1 score: 0.98829, speed: 1.68 step/s
global step 360, epoch: 3, batch: 22, loss: 0.14351, f1 score: 0.98821, speed: 1.57 step/s
eval loss: 7.27132, f1 score: 0.48481, precison: 0.53005, recall: 0.46868
eval loss: 7.10334, f1 score: 0.33454, precison: 0.32406, recall: 0.42970
global step 370, epoch: 3, batch: 32, loss: 0.23289, f1 score: 0.99221, speed: 0.56 step/s
global step 380, epoch: 3, batch: 42, loss: 0.13566, f1 score: 0.99182, speed: 1.66 step/s
global step 390, epoch: 3, batch: 52, loss: 0.31035, f1 score: 0.99115, speed: 1.66 step/s
global step 400, epoch: 3, batch: 62, loss: 0.31810, f1 score: 0.98940, speed: 1.67 step/s
eval loss: 7.33294, f1 score: 0.47639, precison: 0.53946, recall: 0.45930
eval loss: 7.20191, f1 score: 0.33663, precison: 0

[2022-12-29 12:42:27,476] [    INFO] - Configuration saved in ernie2.0_ckpt_2/config.json
[2022-12-29 12:42:40,149] [    INFO] - tokenizer config file saved in ernie2.0_ckpt_2/tokenizer_config.json
[2022-12-29 12:42:40,153] [    INFO] - Special tokens file saved in ernie2.0_ckpt_2/special_tokens_map.json


global step 970, epoch: 6, batch: 125, loss: 0.06501, f1 score: 0.99287, speed: 0.33 step/s
global step 980, epoch: 6, batch: 135, loss: 0.19972, f1 score: 0.99013, speed: 1.66 step/s
global step 990, epoch: 6, batch: 145, loss: 0.11650, f1 score: 0.99009, speed: 1.61 step/s
global step 1000, epoch: 6, batch: 155, loss: 0.07342, f1 score: 0.99112, speed: 1.58 step/s
eval loss: 7.70304, f1 score: 0.48682, precison: 0.53351, recall: 0.47065
eval loss: 7.69689, f1 score: 0.35761, precison: 0.33537, recall: 0.46318
global step 1010, epoch: 6, batch: 165, loss: 0.04741, f1 score: 0.98182, speed: 0.55 step/s
global step 1020, epoch: 7, batch: 6, loss: 0.05366, f1 score: 0.98776, speed: 1.67 step/s
global step 1030, epoch: 7, batch: 16, loss: 0.10682, f1 score: 0.99035, speed: 1.73 step/s
global step 1040, epoch: 7, batch: 26, loss: 0.11000, f1 score: 0.99068, speed: 1.58 step/s
eval loss: 7.67372, f1 score: 0.48071, precison: 0.53462, recall: 0.46123
eval loss: 7.60245, f1 score: 0.32904, pr

In [16]:
# 模型加载
# 加载验证集上效果最好的模型参数
model.set_dict(paddle.load('ernie2.0_ckpt_2/model_state.pdparams'))

# 加载之前训练好的模型参数
# model.set_dict(paddle.load('/home/aistudio/work/model_state.pdparams'))

# 模型验证
print("ERNIE 3.0 在20分类验证集的最佳表现：", end= " ")
results1 = evaluate(model, criterion, metric, valid_data_loader, label_vocab)
print("ERNIE 2.0 在20分类知乎验证集的最佳表现：", end= " ")
results = evaluate(model, criterion, metric, validZhihu_data_loader, label_vocab)

ERNIE 3.0 在20分类验证集的最佳表现： eval loss: 7.59323, f1 score: 0.49040, precison: 0.53397, recall: 0.47277
ERNIE 2.0 在20分类知乎验证集的最佳表现： eval loss: 7.68647, f1 score: 0.35710, precison: 0.31020, recall: 0.47708


In [17]:
def result2tsv(result1,filepath):
    validData = pd.read_csv(filepath,sep='\t')
    dictvalidT =validData.to_dict("list")

    validPred = {}
    validPred["Argument ID"] = dictvalidT["Argument ID"]
    validPred["sentence"] = dictvalidT["sentence"]

    for x in label_vocab:
        validPred[x] = []
    
    for x in range(len(results1)):
        types = results1[x].split(",")
        if types == ['']:
            for y in label_vocab:
                validPred[y].append(0)
        else:  
            for z in label_vocab:
                if z in types:
                    validPred[z].append(1)
                else:
                    validPred[z].append(0)
    validData = pd.read_csv(filepath,sep='\t')
    for x in label_vocab:
        for y in range(len(validData[x])):
            validData[x].iloc[y] = validPred[x][y]
    
    validData.drop(columns=["sentence"],inplace=True)
    return validData

In [18]:
%cd /home/aistudio/data

/home/aistudio/data


In [19]:
import pandas as pd
valid = result2tsv(results1,"validation.tsv")
valid.to_csv('validERNIE2.0.tsv',columns=valid.columns.tolist(),
            sep='\t',
            index=False)

validzhihu = result2tsv(results,"zhihu_validation.tsv")
validzhihu.to_csv('validERNIE2.0zhihu.tsv',columns=validzhihu.columns.tolist(),
            sep='\t',
            index=False)

In [57]:
# 预测函数，对测试集结果进行预测
def predict(model, criterion, metric, data_loader, label_vocab, if_return_results=True):
    model.eval()
    results = []
    for batch in data_loader:
        input_ids, token_type_ids = batch['input_ids'], batch['token_type_ids']
        logits = model(input_ids, token_type_ids)
        probs = logits
        if if_return_results:
            probs = probs.tolist()
            for prob in probs:
                result = []
                for c, pred in enumerate(prob):
                    if pred > 0:
                        result.append(label_vocab[c])
                        # result.append(str(c))
                results.append(','.join(result))
    if if_return_results:
        return results
    else:
        return f1_score


In [68]:
def testOutput(results2):
    testData = pd.read_csv("test.tsv",sep='\t')
    dicttestT =testData.to_dict("list")

    testPred = {}
    testPred["Argument ID"] = dicttestT["Argument ID"]
    testPred["sentence"] = dicttestT["sentence"]

    for x in label_vocab:
        testPred[x] = []

    for x in range(len(results2)):
        types = results2[x].split(",")
        if types == ['']:
            for y in label_vocab:
                testPred[y].append(0)
        else:  
            for z in label_vocab:
                if z in types:
                    testPred[z].append(1)
                else:
                    testPred[z].append(0)
    testPredD = pd.DataFrame.from_dict(testPred)
    testPredD.drop(columns=["sentence"],inplace=True)

    return testPredD

In [58]:
results2 = predict(model, criterion, metric, test_data_loader, label_vocab)


In [69]:
test = testOutput(results2)
test.to_csv('testErnie2.0.tsv',columns=test.columns.tolist(),
            sep='\t',
            index=False)